In [1]:
import torch
import copy
import json
import tqdm
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from pandas import DataFrame
import torch.nn.functional as F

In [2]:
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
torch.cuda.manual_seed(53113)

In [3]:
data_path = 'bridge_data_14-16.json'
df = pd.read_json(data_path)

In [4]:
trX = df
n_train = int(trX.shape[0]*0.8)

In [5]:
for c in trX.columns:
    trX[c] = pd.to_numeric(trX[c]) 

In [6]:
trX[trX.isna().any(axis=1)]
trX = trX.dropna(axis=0,how='any')

In [7]:
train_labels = torch.from_numpy(trX['label'][:n_train].values)
test_labels = torch.from_numpy(trX['label'][n_train:].values)

trX = trX.drop(['label'],axis=1)

train_features = torch.tensor(trX[:n_train].values, dtype=torch.float32)
test_features = torch.tensor(trX[n_train:].values, dtype=torch.float32) 

In [8]:
batch_size=200
learning_rate=0.01
epochs=20
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(290, 200), nn.ReLU(True))
        self.layer2 = nn.Sequential(nn.Linear(200, 100), nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(100, 52), nn.ReLU(True))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer4(x)
        return x

In [10]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)
# criteon = torch.nn.MSELoss().to(device)

In [11]:
from torch.autograd import Variable
for epoch in range(epochs):
    flag = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        
        loss = criteon(logits, target.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
            
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()
        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/483794 (0%)]	Loss: 3.951149
Train Epoch: 0 [20000/483794 (4%)]	Loss: 3.949410
Train Epoch: 0 [40000/483794 (8%)]	Loss: 3.952357
Train Epoch: 0 [60000/483794 (12%)]	Loss: 3.953836
Train Epoch: 0 [80000/483794 (17%)]	Loss: 3.946809
Train Epoch: 0 [100000/483794 (21%)]	Loss: 3.950262
Train Epoch: 0 [120000/483794 (25%)]	Loss: 3.943608
Train Epoch: 0 [140000/483794 (29%)]	Loss: 3.944043
Train Epoch: 0 [160000/483794 (33%)]	Loss: 3.950314
Train Epoch: 0 [180000/483794 (37%)]	Loss: 3.949552
Train Epoch: 0 [200000/483794 (41%)]	Loss: 3.946473
Train Epoch: 0 [220000/483794 (45%)]	Loss: 3.940724
Train Epoch: 0 [240000/483794 (50%)]	Loss: 3.951104
Train Epoch: 0 [260000/483794 (54%)]	Loss: 3.942098
Train Epoch: 0 [280000/483794 (58%)]	Loss: 3.946273
Train Epoch: 0 [300000/483794 (62%)]	Loss: 3.948582
Train Epoch: 0 [320000/483794 (66%)]	Loss: 3.941524
Train Epoch: 0 [340000/483794 (70%)]	Loss: 3.942764
Train Epoch: 0 [360000/483794 (74%)]	Loss: 3.936554
Train Epoch: 0 [380000/4

Train Epoch: 6 [40000/483794 (8%)]	Loss: 2.750144
Train Epoch: 6 [60000/483794 (12%)]	Loss: 2.731101
Train Epoch: 6 [80000/483794 (17%)]	Loss: 2.685533
Train Epoch: 6 [100000/483794 (21%)]	Loss: 2.718900
Train Epoch: 6 [120000/483794 (25%)]	Loss: 2.767987
Train Epoch: 6 [140000/483794 (29%)]	Loss: 2.686723
Train Epoch: 6 [160000/483794 (33%)]	Loss: 2.797683
Train Epoch: 6 [180000/483794 (37%)]	Loss: 2.758039
Train Epoch: 6 [200000/483794 (41%)]	Loss: 2.686089
Train Epoch: 6 [220000/483794 (45%)]	Loss: 2.603021
Train Epoch: 6 [240000/483794 (50%)]	Loss: 2.602545
Train Epoch: 6 [260000/483794 (54%)]	Loss: 2.700636
Train Epoch: 6 [280000/483794 (58%)]	Loss: 2.627689
Train Epoch: 6 [300000/483794 (62%)]	Loss: 2.551027
Train Epoch: 6 [320000/483794 (66%)]	Loss: 2.268870
Train Epoch: 6 [340000/483794 (70%)]	Loss: 2.575069
Train Epoch: 6 [360000/483794 (74%)]	Loss: 2.597525
Train Epoch: 6 [380000/483794 (79%)]	Loss: 2.345355
Train Epoch: 6 [400000/483794 (83%)]	Loss: 2.402321
Train Epoch: 6 [

Train Epoch: 12 [60000/483794 (12%)]	Loss: 1.853837
Train Epoch: 12 [80000/483794 (17%)]	Loss: 1.649484
Train Epoch: 12 [100000/483794 (21%)]	Loss: 2.005105
Train Epoch: 12 [120000/483794 (25%)]	Loss: 1.829851
Train Epoch: 12 [140000/483794 (29%)]	Loss: 1.761358
Train Epoch: 12 [160000/483794 (33%)]	Loss: 1.718154
Train Epoch: 12 [180000/483794 (37%)]	Loss: 1.967259
Train Epoch: 12 [200000/483794 (41%)]	Loss: 1.822870
Train Epoch: 12 [220000/483794 (45%)]	Loss: 1.804526
Train Epoch: 12 [240000/483794 (50%)]	Loss: 1.711426
Train Epoch: 12 [260000/483794 (54%)]	Loss: 1.808494
Train Epoch: 12 [280000/483794 (58%)]	Loss: 1.672233
Train Epoch: 12 [300000/483794 (62%)]	Loss: 1.826897
Train Epoch: 12 [320000/483794 (66%)]	Loss: 1.848793
Train Epoch: 12 [340000/483794 (70%)]	Loss: 1.848888
Train Epoch: 12 [360000/483794 (74%)]	Loss: 1.871964
Train Epoch: 12 [380000/483794 (79%)]	Loss: 1.716485
Train Epoch: 12 [400000/483794 (83%)]	Loss: 1.415405
Train Epoch: 12 [420000/483794 (87%)]	Loss: 1.63

Train Epoch: 18 [40000/483794 (8%)]	Loss: 1.524388
Train Epoch: 18 [60000/483794 (12%)]	Loss: 1.511725
Train Epoch: 18 [80000/483794 (17%)]	Loss: 1.665300
Train Epoch: 18 [100000/483794 (21%)]	Loss: 1.403109
Train Epoch: 18 [120000/483794 (25%)]	Loss: 1.592749
Train Epoch: 18 [140000/483794 (29%)]	Loss: 1.511771
Train Epoch: 18 [160000/483794 (33%)]	Loss: 1.796543
Train Epoch: 18 [180000/483794 (37%)]	Loss: 1.687317
Train Epoch: 18 [200000/483794 (41%)]	Loss: 1.456850
Train Epoch: 18 [220000/483794 (45%)]	Loss: 1.511163
Train Epoch: 18 [240000/483794 (50%)]	Loss: 1.566865
Train Epoch: 18 [260000/483794 (54%)]	Loss: 1.411759
Train Epoch: 18 [280000/483794 (58%)]	Loss: 1.674953
Train Epoch: 18 [300000/483794 (62%)]	Loss: 1.524887
Train Epoch: 18 [320000/483794 (66%)]	Loss: 1.416426
Train Epoch: 18 [340000/483794 (70%)]	Loss: 1.496888
Train Epoch: 18 [360000/483794 (74%)]	Loss: 1.284989
Train Epoch: 18 [380000/483794 (79%)]	Loss: 1.495210
Train Epoch: 18 [400000/483794 (83%)]	Loss: 1.4589

In [12]:
# 保存训练的模型：
torch.save(net.state_dict(),"bridge_agent_v1.pth")